In [16]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import scipy.spatial.distance as distance
import pandas as pd
import numpy as np

distmodels = ["BasicDistanceModel", "GitDiffDistModel", 
              "MossDistModel", "PatientDiffDistModel"]
scoremodels = ["SimpleScoreModel", "LongestScoreModel", 
               "ShortestScoreModel", "InEdgesScoreModel",
               "OutEdgesScoreModel", "SizeScoreModel", 
               "TimeWeightedScoreModel"]

models_inedges = ['InEdgesScoreModel_' + dm for dm in distmodels]

In [3]:
df = pd.read_csv('test4.csv')
df['buggy'] = (df['rating'] > 0)*1
df['is_train'] = np.random.uniform(0, 1, len(df)) <= 0.7
train, test = df[df['is_train']==True], df[df['is_train']==False]

In [75]:
features = df.columns[1:]
features = np.array(['SizeScoreModel_BasicDistanceModel'])
# features = np.array(['SizeScoreModel_BasicDistanceModel','OutEdgesScoreModel_GitDiffDistModel','ShortestScoreModel_GitDiffDistModel','OutEdgesScoreModel_PatientDiffDistModel','SimpleScoreModel_GitDiffDistModel'])
# features = np.array(['SimpleScoreModel_BasicDistanceModel','SimpleScoreModel_MossDistModel','LongestScoreModel_BasicDistanceModel','ShortestScoreModel_BasicDistanceModel','OutEdgesScoreModel_BasicDistanceModel'])
exclude = ['rating', 'is_train', 'buggy',
           'SizeScoreModel_GitDiffDistModel', 
           'SizeScoreModel_MossDistModel',
           'SizeScoreModel_PatientDiffDistModel']
exclude.extend(models_inedges)
for ex in exclude:
    features = features[features != ex]
for feature in features:
    print feature

SizeScoreModel_BasicDistanceModel


In [81]:
clf = RandomForestClassifier(n_jobs=8)
y, _ = pd.factorize(train['buggy'])
clf.fit(train[features], y)

preds = clf.predict(test[features])
actual, _ = pd.factorize(test['buggy'])
def percent_bugs_ided(preds, actual):
    correct = 0.
    for i in xrange(0, len(preds)):
        if actual[i] == 1 and preds[i] == 1:
            correct += 1
    return correct/sum(actual)
def percent_normal_ided(preds, actual):
    correct = 0.
    for i in xrange(0, len(preds)):
        if actual[i] == 0 and preds[i] == 0:
            correct += 1
#     return len(actual) - correct
    return correct/(len(actual) - sum(actual))
def overall_accuracy(preds, actual):
    num_correct = np.sum(preds == actual)
    return num_correct*1./len(preds)

num_correct = np.sum(preds == actual)

print percent_bugs_ided(preds, actual)
print percent_normal_ided(preds, actual)
print overall_accuracy(preds, actual)


0.0129310344828
0.999858232442
0.993408636376


In [78]:
impt = sorted(zip(features, clf.feature_importances_), key = lambda x: -x[1])
def change_name(name):
    return name \
        .replace("SizeScoreModel_BasicDistanceModel", "SizeScoreModel") \
        .replace("_", "\_") \
        .replace("Git", "Myer") \
        .replace("ScoreModel", "") \
        .replace("DistModel", "Dist") \
        .replace("BasicDistanceModel", "UnitDist") \
        .replace("TimeWeighted", "TimeWeightAvePath") \
        .replace("Shortest", "MinPath") \
        .replace("Longest", "MaxPath") \
        .replace("Size", "PatchLen") \
        .replace("Edges", "EdgesSum") \
        .replace("Simple", "AvePath")
    
for line in impt:
    print "%s & %f \\\\" % \
        (change_name(line[0]), 
         line[1])

PatchLen & 1.000000 \\


In [70]:
impt = sorted(zip(features, clf.feature_importances_), key = lambda x: -x[1])
models = zip(*impt)[0]
names = map(lambda s: "'%s'" % s, models)
','.join(names)

"'SizeScoreModel_BasicDistanceModel','OutEdgesScoreModel_GitDiffDistModel','ShortestScoreModel_GitDiffDistModel','OutEdgesScoreModel_PatientDiffDistModel','SimpleScoreModel_GitDiffDistModel','ShortestScoreModel_PatientDiffDistModel','TimeWeightedScoreModel_PatientDiffDistModel','SimpleScoreModel_PatientDiffDistModel','TimeWeightedScoreModel_BasicDistanceModel','TimeWeightedScoreModel_MossDistModel','LongestScoreModel_GitDiffDistModel','TimeWeightedScoreModel_GitDiffDistModel','OutEdgesScoreModel_MossDistModel','LongestScoreModel_PatientDiffDistModel','ShortestScoreModel_MossDistModel','LongestScoreModel_MossDistModel','SimpleScoreModel_BasicDistanceModel','SimpleScoreModel_MossDistModel','LongestScoreModel_BasicDistanceModel','ShortestScoreModel_BasicDistanceModel','OutEdgesScoreModel_BasicDistanceModel'"

In [7]:
sum(actual)*1./len(actual)

0.011017115876450915

In [48]:
zip(*impt)

[('SizeScoreModel_BasicDistanceModel',
  'OutEdgesScoreModel_GitDiffDistModel',
  'OutEdgesScoreModel_PatientDiffDistModel',
  'ShortestScoreModel_PatientDiffDistModel',
  'TimeWeightedScoreModel_GitDiffDistModel',
  'ShortestScoreModel_GitDiffDistModel',
  'SimpleScoreModel_GitDiffDistModel',
  'TimeWeightedScoreModel_MossDistModel',
  'LongestScoreModel_PatientDiffDistModel',
  'LongestScoreModel_GitDiffDistModel',
  'SimpleScoreModel_PatientDiffDistModel',
  'TimeWeightedScoreModel_PatientDiffDistModel',
  'TimeWeightedScoreModel_BasicDistanceModel',
  'ShortestScoreModel_MossDistModel',
  'OutEdgesScoreModel_MossDistModel',
  'LongestScoreModel_MossDistModel',
  'SimpleScoreModel_MossDistModel',
  'SimpleScoreModel_BasicDistanceModel',
  'LongestScoreModel_BasicDistanceModel',
  'ShortestScoreModel_BasicDistanceModel',
  'OutEdgesScoreModel_BasicDistanceModel'),
 (0.096456986331355077,
  0.071530542516494364,
  0.068879113480828674,
  0.066720901183964204,
  0.064443365797795499,
 

In [114]:
df.columns[10:16]

Index([u'ShortestScoreModel_GitDiffDistModel',
       u'InEdgesScoreModel_GitDiffDistModel',
       u'OutEdgesScoreModel_GitDiffDistModel',
       u'SizeScoreModel_GitDiffDistModel',
       u'TimeWeightedScoreModel_GitDiffDistModel',
       u'SimpleScoreModel_MossDistModel'],
      dtype='object')

In [79]:
df.columns[1:22]

Index([u'SimpleScoreModel_BasicDistanceModel',
       u'LongestScoreModel_BasicDistanceModel',
       u'ShortestScoreModel_BasicDistanceModel',
       u'InEdgesScoreModel_BasicDistanceModel',
       u'OutEdgesScoreModel_BasicDistanceModel',
       u'SizeScoreModel_BasicDistanceModel',
       u'TimeWeightedScoreModel_BasicDistanceModel',
       u'SimpleScoreModel_GitDiffDistModel',
       u'LongestScoreModel_GitDiffDistModel',
       u'ShortestScoreModel_GitDiffDistModel',
       u'InEdgesScoreModel_GitDiffDistModel',
       u'OutEdgesScoreModel_GitDiffDistModel',
       u'SizeScoreModel_GitDiffDistModel',
       u'TimeWeightedScoreModel_GitDiffDistModel',
       u'SimpleScoreModel_MossDistModel', u'LongestScoreModel_MossDistModel',
       u'ShortestScoreModel_MossDistModel', u'InEdgesScoreModel_MossDistModel',
       u'OutEdgesScoreModel_MossDistModel', u'SizeScoreModel_MossDistModel',
       u'TimeWeightedScoreModel_MossDistModel'],
      dtype='object')

In [26]:
print len(df)
print sum(df['buggy'])

118605
1307
